In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ============================================
# 1) Instalar e importar bibliotecas
# ============================================
!pip install folium geopandas shapely openpyxl

import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed

# ============================================
# 2) Conectar ao Google Drive
# ============================================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ============================================
# 3) Carregar dados de Excel (pontos) e criar buffers
# ============================================
dados_projeto_path = "/content/drive/MyDrive/petroleo/01_Petrole_Colab/dados_projeto_petroleo.xlsx"
df_dados = pd.read_excel(dados_projeto_path)

df_dados['geometry'] = df_dados.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
gdf_dados = gpd.GeoDataFrame(df_dados, geometry='geometry', crs='EPSG:4326')

buffer_distance = 5000
gdf_dados = gdf_dados.to_crs(epsg=3857)
gdf_dados['buffer'] = gdf_dados.geometry.buffer(buffer_distance)
gdf_dados = gdf_dados.to_crs(epsg=4326)

buffer_50km = 50000
poco_fza = gdf_dados[gdf_dados['Nome'].str.contains("FZA-M-59", na=False, case=False)].copy()
if not poco_fza.empty:
    print("Poço FZA-M-59 encontrado! Criando buffer...")
    poco_fza = poco_fza.to_crs(epsg=3857)
    poco_fza['buffer'] = poco_fza.geometry.buffer(buffer_50km)
    poco_fza = poco_fza.to_crs(epsg=4326)
else:
    print("⚠️ Poço FZA-M-59 não encontrado na base de dados!")

# ============================================
# 4) Definir caminhos dos shapefiles
# ============================================
shapefiles = {
    "Bacias Sedimentares": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/BaciasSedimentares/bacias_gishub_db.shp",
    "Biomas 250mil": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/Biomas_250mil/lm_bioma_250.shp",
    "Áreas Protegidas": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/cat60_protected_area_WGS84_v2/cat60_protected_area_WGS84_v2.shp",
    "Sistemas Costeiros e Marinhos": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/Sistema_Costeiro_Marinho_250mil/Sistema_Costeiro_Marinho.shp",
    "Biomas 5000mil": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/Biomas_5000mil/Biomas5000.shp",
    "ZCM - Áreas Prioritárias (2ª Atualização)": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/zcm/ZCM_2a_atualizacao.shp",
    "Amazônia - Áreas Prioritárias (2ª Atualização)": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/amazonia/Amazonia_2a_atualizacao.shp",
    "Cerrado - Áreas Prioritárias (2ª Atualização)": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/cerrado_pantanal/Cerrado_Pantanal_2a_atualizacao.shp",
    "Caatinga - Áreas Prioritárias (2ª Atualização)": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/caatinga/Caatinga_2a_atualizacao.shp",
    "Blocos em Oferta - 5º Ciclo": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/OPC_Blocos_em_Oferta_5_Ciclo/opc_blocos_em_oferta.shp",
    "Bloco Exploratório": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/bloco/block_shp_geoanp.shp"
}

# ============================================
# 5) Parâmetros de simplificação
# ============================================
TOLERANCE = 200
SIMPLIFY = True

# ============================================
# 6) Função para ler e preparar cada shapefile
# ============================================
def read_and_prepare_shapefile(layer_name, shp_path):
    try:
        gdf_layer = gpd.read_file(shp_path)
        if gdf_layer.crs is None:
            print(f"[Aviso] '{layer_name}' não possui CRS definido. Atribuindo EPSG:4326.")
            gdf_layer.set_crs(epsg=4326, inplace=True)

        if SIMPLIFY:
            gdf_layer = gdf_layer.to_crs(epsg=3857)
            gdf_layer['geometry'] = gdf_layer.geometry.simplify(
                tolerance=TOLERANCE,
                preserve_topology=True
            )
            gdf_layer = gdf_layer.to_crs(epsg=4326)

        for col in gdf_layer.select_dtypes(include=['datetime64[ns]']).columns:
            gdf_layer[col] = gdf_layer[col].astype(str)

        return (layer_name, gdf_layer)
    except Exception as e:
        print(f"Erro ao carregar {layer_name} ({shp_path}): {e}")
        return (layer_name, None)

# ============================================
# 7) Ler shapefiles em paralelo
# ============================================
shapefile_results = []
MAX_WORKERS = 4
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    future_to_layer = {
        executor.submit(read_and_prepare_shapefile, name, path): name
        for name, path in shapefiles.items()
    }
    for future in as_completed(future_to_layer):
        layer_name = future_to_layer[future]
        try:
            result = future.result()
            shapefile_results.append(result)
        except Exception as e:
            print(f"Exceção ao processar {layer_name}: {e}")

# ============================================
# 8) Criar o mapa base com múltiplas opções de fundo
# ============================================
m = folium.Map(location=[-2.5, -45], zoom_start=6, tiles=None)
# Adicionar camadas de fundo
folium.TileLayer("OpenStreetMap", name="OpenStreetMap", control=True).add_to(m)
folium.TileLayer("CartoDB Positron", name="CartoDB Positron", control=True).add_to(m)

# ============================================
# 9) Camadas dos PONTOS e buffers
# ============================================
descricoes_unicas = df_dados['Descrição'].unique()
layers_dict = {}
for descricao in descricoes_unicas:
    layers_dict[descricao] = folium.FeatureGroup(name=f"{descricao}", show=False)
    dados_descricao = gdf_dados[gdf_dados['Descrição'] == descricao]

    for _, row in dados_descricao.iterrows():
        folium.GeoJson(
            row.buffer,
            style_function=lambda x, color=row['Cor']: {
                "color": color,
                "weight": 1,
                "fillOpacity": 0.2
            },
            tooltip=f"Buffer 5km: {row['Nome']}"
        ).add_to(layers_dict[descricao])

    for _, row in dados_descricao.iterrows():
        popup_text = f"""
        <div style="font-family: Arial, sans-serif; font-size: 12px;">
            <strong>Descrição:</strong> {row['Descrição']}<br>
            <strong>Nome:</strong> {row['Nome']}
        </div>
        """
        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=6,
            color=row['Cor'],
            fill=True,
            fill_color=row['Cor'],
            fill_opacity=0.9,
            popup=folium.Popup(popup_text, max_width=300),
        ).add_to(layers_dict[descricao])

    layers_dict[descricao].add_to(m)

# ============================================
# 10) Buffer de 50 km do poço FZA-M-59
# ============================================
if not poco_fza.empty:
    layer_fza = folium.FeatureGroup(name="FZA-M-59 - Buffer 50km", show=False)
    for _, row in poco_fza.iterrows():
        folium.GeoJson(
            row.buffer,
            style_function=lambda x: {
                "color": "gray",
                "weight": 4,
                "fillOpacity": 0.3,
                "dashArray": "5,5"
            },
            tooltip=f"Buffer 50km: {row['Nome']}"
        ).add_to(layer_fza)

    folium.CircleMarker(
        location=[poco_fza.geometry.y.iloc[0], poco_fza.geometry.x.iloc[0]],
        radius=10,
        color="black",
        fill=True,
        fill_color="gray",
        fill_opacity=1,
        popup="FZA-M-59 (Poço Destacado)"
    ).add_to(layer_fza)

    layer_fza.add_to(m)

# ============================================
# 11) Definir mapeamento de prioridades e cores (incluindo cinza para 'sem classificação')
# ============================================
synonyms = {
    'extremamente alta': 'extremamente alta',
    'muito alta': 'muito alta',
    'muita alta': 'muito alta',
    'alta': 'alta'
}

priority_colors = {
    'extremamente alta': '#FF0000',  # Vermelho
    'muito alta': '#FFA500',        # Laranja
    'alta': '#FFFF00'               # Amarelo
}

def style_priority(feature):
    props = feature['properties']
    original_value = props.get('Prior_acao') or props.get('PRIO') or 'Sem classificacao'
    original_value = str(original_value).strip().lower()
    valor_padronizado = synonyms.get(original_value, 'sem classificacao')
    color = priority_colors.get(valor_padronizado, '#B8B8B8')  # Cinza para "sem classificacao"
    return {
        "fillColor": color,
        "color": "black",
        "weight": 1,
        "fillOpacity": 0.5
    }

# ============================================
# 12) Adicionar shapefiles ao mapa
# ============================================
priority_layers = [
    "Amazônia - Áreas Prioritárias (2ª Atualização)",
    "Cerrado - Áreas Prioritárias (2ª Atualização)",
    "Caatinga - Áreas Prioritárias (2ª Atualização)",
    "ZCM - Áreas Prioritárias (2ª Atualização)"
]

for layer_name, gdf_layer in shapefile_results:
    if gdf_layer is None:
        continue

    layer_fg = folium.FeatureGroup(name=layer_name, show=False)

    if layer_name in priority_layers:
        folium.GeoJson(
            gdf_layer,
            style_function=style_priority,
            tooltip=folium.GeoJsonTooltip(
                fields=[c for c in gdf_layer.columns if c.lower() != 'geometry'],
                aliases=[c for c in gdf_layer.columns if c.lower() != 'geometry']
            )
        ).add_to(layer_fg)
    else:
        all_cols = [c for c in gdf_layer.columns if c.lower() != 'geometry']
        folium.GeoJson(
            gdf_layer,
            style_function=lambda x: {
                "color": "blue",
                "weight": 1,
                "fillOpacity": 0.2
            },
            tooltip=folium.GeoJsonTooltip(fields=all_cols, aliases=all_cols)
        ).add_to(layer_fg)

    layer_fg.add_to(m)

# ============================================
# 13) Ajustar zoom do mapa e exibir
# ============================================
m.fit_bounds([
    [min(df_dados['Latitude']), min(df_dados['Longitude'])],
    [max(df_dados['Latitude']), max(df_dados['Longitude'])]
])

folium.LayerControl().add_to(m)
m

In [63]:
# ============================================
# 1) Instalar e importar bibliotecas
# ============================================
!pip install folium geopandas shapely openpyxl

import folium
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor, as_completed

# ============================================
# 2) Conectar ao Google Drive
# ============================================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# ============================================
# 3) Carregar dados de Excel (pontos) e criar buffers
# ============================================
dados_projeto_path = "/content/drive/MyDrive/petroleo/01_Petrole_Colab/dados_projeto_petroleo.xlsx"
df_dados = pd.read_excel(dados_projeto_path)

df_dados['geometry'] = df_dados.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
gdf_dados = gpd.GeoDataFrame(df_dados, geometry='geometry', crs='EPSG:4326')

buffer_distance = 5000
gdf_dados = gdf_dados.to_crs(epsg=3857)
gdf_dados['buffer'] = gdf_dados.geometry.buffer(buffer_distance)
gdf_dados = gdf_dados.to_crs(epsg=4326)

buffer_50km = 50000
poco_fza = gdf_dados[gdf_dados['Nome'].str.contains("FZA-M-59", na=False, case=False)].copy()
if not poco_fza.empty:
    print("Poço FZA-M-59 encontrado! Criando buffer...")
    poco_fza = poco_fza.to_crs(epsg=3857)
    poco_fza['buffer'] = poco_fza.geometry.buffer(buffer_50km)
    poco_fza = poco_fza.to_crs(epsg=4326)
else:
    print("⚠️ Poço FZA-M-59 não encontrado na base de dados!")

# ============================================
# 4) Definir caminhos dos shapefiles
# ============================================
shapefiles = {
    "Bacias Sedimentares": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/BaciasSedimentares/bacias_gishub_db.shp",
    "Biomas 250mil": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/Biomas_250mil/lm_bioma_250.shp",
    "Áreas Protegidas": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/cat60_protected_area_WGS84_v2/cat60_protected_area_WGS84_v2.shp",
    "Sistemas Costeiros e Marinhos": "/content/drive/MyDrive/petroleo/01_Petrole_Colab/Sistema_Costeiro_Marinho_250mil/Sistema_Costeiro_Marinho.shp",
}

# ============================================
# 5) Parâmetros de simplificação
# ============================================
TOLERANCE = 1000  # Aumentado para reduzir tamanho
SIMPLIFY = True

# ============================================
# 6) Função para ler e preparar cada shapefile
# ============================================
def read_and_prepare_shapefile(layer_name, shp_path):
    try:
        gdf_layer = gpd.read_file(shp_path)
        if gdf_layer.crs is None:
            print(f"[Aviso] '{layer_name}' não possui CRS definido. Atribuindo EPSG:4326.")
            gdf_layer.set_crs(epsg=4326, inplace=True)

        if SIMPLIFY:
            gdf_layer = gdf_layer.to_crs(epsg=3857)
            gdf_layer['geometry'] = gdf_layer.geometry.simplify(
                tolerance=TOLERANCE,
                preserve_topology=True
            )
            gdf_layer = gdf_layer.to_crs(epsg=4326)

        # ✅ Converter colunas datetime para string
        for col in gdf_layer.select_dtypes(include=['datetime64[ns]']).columns:
            gdf_layer[col] = gdf_layer[col].astype(str)

        # ✅ Remover colunas desnecessárias (IDs e códigos longos)
        drop_cols = [col for col in gdf_layer.columns if "ID" in col or "Code" in col]
        gdf_layer.drop(columns=drop_cols, inplace=True, errors="ignore")

        return (layer_name, gdf_layer)
    except Exception as e:
        print(f"Erro ao carregar {layer_name} ({shp_path}): {e}")
        return (layer_name, None)
# ============================================
# 7) Ler shapefiles em paralelo
# ============================================
shapefile_results = []
MAX_WORKERS = 2  # Reduzi para evitar sobrecarga
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    future_to_layer = {
        executor.submit(read_and_prepare_shapefile, name, path): name
        for name, path in shapefiles.items()
    }
    for future in as_completed(future_to_layer):
        layer_name = future_to_layer[future]
        try:
            result = future.result()
            shapefile_results.append(result)
        except Exception as e:
            print(f"Exceção ao processar {layer_name}: {e}")

# ============================================
# 8) Criar e salvar o mapa em formato reduzido
# ============================================
m = folium.Map(location=[-2.5, -45], zoom_start=6, tiles="CartoDB Positron")

for layer_name, gdf_layer in shapefile_results:
    if gdf_layer is None:
        continue

    folium.GeoJson(
        gdf_layer,
        style_function=lambda x: {
            "color": "blue",
            "weight": 1,
            "fillOpacity": 0.2
        }
    ).add_to(m)

# Salvar em formato reduzido para o GitHub
map_file = "/content/drive/MyDrive/petroleo/mapa_reduzido.html"
m.save(map_file)
print(f"Mapa salvo em: {map_file}")


Mounted at /content/drive
Poço FZA-M-59 encontrado! Criando buffer...
Mapa salvo em: /content/drive/MyDrive/petroleo/mapa_reduzido.html
